#Token Ring

##Постановка задачи

Задача: Построение простой модели сетевого протокола TokenRing и исследовании его свойств

Этапы:
+ Реализовать модель протокола
+ Исследовать пропускную способность сети (throughput) и характерное время задержки (latency) в зависимости от количества узлов N и количества пакетов P
+ Проанализировать полученные результаты
+ Оптимизировать throughput или latency и исследовать влияние оптимизаций для одного режима на весь спектр режимов.


## Определения

+ Характерное время задержки (latency) - промежуток времени с момента отправки токена с ноды-отправителя до момента получения на ноде-получателе. 

+ Пропускную способность сети (throughput) - количество токенов прошедших через ноду за единицу времени.

+ Максимальное количество токенов в системе - размер очереди * количество нод (нод коннекторов), то есть совокупный размер очередей.



## Реализация

Протокол реализован на взаимодействии Node через NodeConnector, представляющий из себя блокирующую очередь (java.util.concurrent).
Ноды передают далее по кругу пакет - токен, содержащий id и вспомогательное поле за замера времени задержки.

При получении токена - нода его обрабатывает и передаёт дальше, не выбрасывая из системы.

На всех нодах кроме одной пустые консьюмеры, нулевая же обрабатывает токен и получает.

Ноды запускаются в разных потоках.


## Описание эспериментальной установки

Ноутбук DELL:
+ Intel® Core™ i5-8300H CPU @ 2.30GHz × 8 
+ RAM: 8Gb DDR3
+ OS: Ubuntu 20.04.2 LTS

## Постановка эксперимента

Эксперимент проводится с помощью библиотеки для модульного тестирования Junit. 

Предварительно проводится "прогрев кода" - работа протокола на 2х нода с 2мя токенами в течение 5 секунд.

Для latency - единицы измерения nanoseconds. Время замеряется с помощью System.nanotime().

Для throughput - единицы измерения количество токенов прошедших за 0.5 секунды.

Как сказано выше, для замера latency и throughput используется только одна нода, соответственно все замеры - это значение за полный круг. 

Мы можем не считывать данные со всех нод, так как все сообщения проходят в определнном порядке, который не нарушится никогда, соответственно, если какой-то токен задержится на какой-то ноде, то он задержится и при прохождении всего круга.

## Определение максимального количества нод для сбора данных


Проведём эксперимент по определению количества нод, доступных для проведения честного эксперимента - максимальное возможное количество потоков, которое мы можем использовать в эксперименте без потери валидности результатов.

Для этого запустим протокол на разном количестве нод.

![](https://drive.google.com/uc?export=view&id=1lem9aaQ_yon7vFOOt__8J7LBFsEZsRAS) ![](https://drive.google.com/uc?export=view&id=1--qKRrUbX9I9tPh3JMz8dgmG3mnZlBOQ)

Заметим, что на количестве нод 5, 6, 7 появляется больший разброс по данным, это связано с тем, что в некторые моменты управление процессами передается системным приложениям.

В дальнейшем будем исследовать только 2, 3, 4 ноды.

## Определение зависимости latency и throughput от размера очереди.

Далее представлены зависимости latency и throughput от размера очереди при разном количестве токенов в системе (нагрузке на систему).

Размер очереди варируется от 2 до количества нод + 2.
Рассматриваем три варианта нагрузки.

Low load - количество токенов равно половине количества нод.

Medium load - количество токенов равно количестве нод.

High load - количество токенов равно 0.8 от 2 * количество нод.
То есть больше количества нод и меньше макисмального количества, так же подходит для любого количества нод с очередью размера 2 и более.

![](https://drive.google.com/uc?export=view&id=1-1dlgCODq4jgYzPKfWugvHsiHh0QulEr)![](https://drive.google.com/uc?export=view&id=1-9n1VTP394Il0jxEnR_VsgAiWllfeihs)![](https://drive.google.com/uc?export=view&id=1-JfvyJpjxOPUvm7i-0xYEYWBVTU5mldf)

С увеличение количества нод увеличивается latency, что теоретически объяснимо - для прохождения полного круга необходимо пройти больше нод.

На первых двух графиках почти нет заметных изменений в latency, увеличение очереди не даёт ускорения, так как количество токенов меньше количества нод, и все токены успевают обработаться без ожидания в очереди.

Заметно небольшое увеличение только для 3 нод при размере очереди 2.

А уже при высокой нагрузке заметно, что при малых размерах очереди увеличилось latency, образуя монотонную обратную зависимость. (Теперь не все токены обрабатываются без ожидания в очереди.)

![](https://drive.google.com/uc?export=view&id=1-NYryDl3EB0rB5sgo08PsnLdIOvLPu7h)![](https://drive.google.com/uc?export=view&id=1-Ns9qdc9vATh3ha4c5kw6UJPJtKBeuUu)![](https://drive.google.com/uc?export=view&id=1-Svb0PsKPxeUCaA0c-OY06SS8ou7be53)

Аналогичную картину мы можем наблюдать и с throughput.
Изменения появляются в основном при повышенной нагрузке, всё по тем же причинам.

Важно заметить, что обработка токенов не занимает много времени, поэтому мы получаем такие результаты на малой и средней нагрузках.

Далее следует более детально изучить зависимость latency и throughput от количества токенов в системе при разных размемрах очереди.

## Определение зависимости latency и throughput от размера очереди для 4х нод

Для этого рассмотрим только один вариант системы с 4 нодами.
Проведём эксперимент для размеров очереди 4, 8, 12 с количеством токенов от 3 до максимального с шагом 4.

![](https://drive.google.com/uc?export=view&id=1zwrmmAfteywZSAnk1trP37l6lMyA9gut)![](https://drive.google.com/uc?export=view&id=1-7AO3M_l5jPoWA0dwujfup5JoLPl3jgT)

По графикам видно, что предположение о эффективности увеличения размера очереди подтвердилось и в этом эксперименте. 
Для одинакового кол-ва токенов latency при большем размере очереди в разы меньше.

Аналогично и для throughput.

Так важно заметить существование оптимального количества токенов в системе для throughput. Которое примерно равно половине максимальной нагрузке, что соответствует заполнености очередей наполовину в среднем. Из-за чего, вероятнее всего, и получается максимум - две соседние ноды смогут обработать весь пулл токенов без торможения остальных.

Проведём дополнительный эксперимент для подтверждения предыдущего и поиска верхрней границы throughput.

![](https://drive.google.com/uc?export=view&id=1-BdrzjCq6urdHKiglzdmLpef98f9d7JP)![](https://drive.google.com/uc?export=view&id=1-AkCA-5Hu-2KFz1SJWKI7EN7JU1Iav4Q)

Характер зависимостей остался таким же.

Так же диапозон оптимальных значений для throughput увеличился, стал более пологим. То есть стал более близок к некой ассимптотической кривой.


## Вывод

Задача: Построение простой модели сетевого протокола TokenRing и исследовании его свойств

Этапы:
В данной работе реализована модель протокола, исследованы throughput и latency в зависимости от количества нод и количества токенов, проанализированы полученные результаты

В результате, получены выводы, что latency прямо пропорциональнно зависит от размера очередей. Благодаря уменьшению времени ожидания нод на передачу токена следующей ноде.

Так же выявлено существование некого оптимального диапозона количества токенов в системе в зависимости от размера очередей, равного половине макисмального количества токенов. Это может быть практически использовано в определении размера очередей для конкретной системы, с известными средними нагрузками в ней.